In [3]:
from pathlib import Path
import pickle
import time
from tqdm import tqdm
from typing import List
import math
import pandas as pd
import os

from projectmir.xmldocument import XMLDocument
from projectmir.xmldoc_child import Identifier
from projectmir.extract_definition import pattern_based_extract_description, pagel_ranking_candidates, kato_ranking_candidates, Definition, evaluate_identifier_definition


In [2]:
p_path = './data/test_latexml/'
p = Path(p_path)
x = list(p.glob('*.html'))
pkl_filename = 'docs_list.pkl'
# if os.path.isfile(p_path + pkl_filename):
#     # load_filename = 'docs_list.pkl'
#     with open(pkl_filename, 'rb') as f:
#         docs_list = pickle.load(f)
# else:
print('create docs_list.pkl...')
documents_IDs = [x_.name[:-5] for x_ in x]
docs_list = [[]]*len(documents_IDs)
for i, documents_ID in tqdm(enumerate(documents_IDs)):
    document_path = p_path + documents_ID + '.html'
    start_time = time.time()
    docs_list[i] = XMLDocument(document_path)
    if i==0:
        break
# save_filename = p_path + pkl_filename
# with open(save_filename, 'wb') as f:
#     pickle.dump(docs_list, f)


0it [00:00, ?it/s]2020-08-27 06:44:11 INFO: Loading these models for language: en (English):
| Processor | Package |
-----------------------
| tokenize  | ewt     |

2020-08-27 06:44:11 INFO: Use device: cpu
2020-08-27 06:44:11 INFO: Loading: tokenize
2020-08-27 06:44:11 INFO: Done loading processors!
2020-08-27 06:44:12 INFO: Loading these models for language: en (English):
| Processor | Package   |
-------------------------
| tokenize  | ewt       |
| pos       | ewt       |
| lemma     | ewt       |
| depparse  | ewt       |
| ner       | ontonotes |

2020-08-27 06:44:12 INFO: Use device: cpu
2020-08-27 06:44:12 INFO: Loading: tokenize
2020-08-27 06:44:12 INFO: Loading: pos
2020-08-27 06:44:13 INFO: Loading: lemma
2020-08-27 06:44:13 INFO: Loading: depparse
2020-08-27 06:44:14 INFO: Loading: ner
2020-08-27 06:44:14 INFO: Done loading processors!
0it [01:01, ?it/s]


create docs_list.pkl...
loaded data.
processing data...
preprocessed document
extract identifiers...
Number of math components is 50
math_text_string.head
<Element head at 0x1308f1220>
math_text_string.head
<Element head at 0x1308f1220>
math_text_string.head
<Element head at 0x1308f1450>
math_text_string.head
<Element head at 0x1308f1310>
math_text_string.head
<Element head at 0x1308f14f0>
math_text_string.head
<Element head at 0x1308f16d0>
math_text_string.head
<Element head at 0x1308f1680>
math_text_string.head
<Element head at 0x1308f14f0>
math_text_string.head
<Element head at 0x1308f1810>
math_text_string.head
<Element head at 0x1308f18b0>
math_text_string.head
<Element head at 0x1308f18b0>
math_text_string.head
<Element head at 0x1308f19a0>
math_text_string.head
<Element head at 0x1308f14f0>
POS tagging...
extract candidate definition...
Starting server with command: java -Xmx16G -cp /Users/kato/GoogleDrive/project-mir/data/resources/stanford-corenlp-4.1.0/* edu.stanford.nlp.pipe

In [1]:
# store variable
# %store docs_list
# load variable
%store -r

In [4]:
doc0 = docs_list[0]
print(doc0.path)

# f = pd.read_csv('./data/test_latexml/identifier_definition.txt')
f = pd.read_csv('./data/test_latexml/identifier_definition_one_char.txt')
gold_identifier = f['Identifier'].to_list()
gold_definition_list = []
for g_list in f['Definition']:
    g_splitted_list = g_list.split(';')
    gold_definition_list.append([g_.rstrip().lstrip() for g_ in g_splitted_list])
gold_definition_list

./data/test_latexml/test_sample.html


[['mass flow rate', 'a mass flow rate'],
 ['inlet temperature', 'an inlet temperature'],
 ['heating rate', 'a heating rate'],
 ['exit temperature', 'the exit temperature'],
 ['outlet flow rate'],
 ['liquid holdup', 'the liquid holdup'],
 ['density', 'the density'],
 ['heat capacity'],
 ['internal energy', 'the internal energy'],
 ['enthalpy', 'the enthalpy'],
 ['internal energy per unit mass', 'the internal energy per unit mass'],
 ['enthalpy per unit mass', 'the enthalpy per unit mass'],
 ['reference temperature', 'a reference temperature'],
 ['value of the enthalpy per unit mass at a reference temperature',
  'the value of the enthalpy per unit mass at a reference temperature'],
 ['time'],
 ['degrees of freedom', 'the degrees of freedom']]

In [14]:
# extract definitions using three methods.
alpha_list = [0.1 * i for i in range(1, 11)]
beta_list = [0.1 * i for i in range(1, 11)]
gamma_list = [0.1 * i for i in range(1, 11)]
eta_list = [0.1 * i for i in range(1, 11)]
sep='\t\t\t'
result_filename = 'test_result.txt'

score_pagel_recall_best = ((0, 0), {})
score_pagel_precision_best = ((0, 0), {})
score_kato_recall_best = ((0, 0), {})
score_kato_precision_best = ((0, 0), {})
score_pagel_recall_best_rank2 = ((0, 0), {})
score_pagel_precision_best_rank2 = ((0, 0), {})
score_kato_recall_best_rank2 = ((0, 0), {})
score_kato_precision_best_rank2 = ((0, 0), {})

with open(result_filename, 'w') as f:
    f.write(sep.join(['id', 'identifier', 'pagel_score', 'pagel', 'kato_score', 'kato']))
    f.write('\n')
for alpha_ in alpha_list:
    for beta_ in [alpha_]:#beta_list:
        for gamma_ in gamma_list:
            for eta_ in eta_list:
                definition_list_pattern = []
                definition_list_pagel = []
                definition_list_kato = []

                with open('test_result.txt', 'a') as f:
                    f.write("===================================")
                    f.write('\n')
                params = {'sigma_d': math.sqrt(12 / math.log(2)), 'sigma_s': 2 / math.sqrt(math.log(2)),
                          'alpha': alpha_, 'beta': beta_, 'gamma': gamma_, 'eta': eta_}
                # print(params)
                with open('test_result.txt', 'a') as f:
                    f.write(f'{params=}')
                    f.write('\n')
                for i, identifier in enumerate(doc0.identifiers):
                    definition_pattern = pattern_based_extract_description(identifier)
                    definition_list_pattern.append(definition_pattern)

                    definition_pagel = pagel_ranking_candidates(identifier, params)
                    definition_list_pagel.append(definition_pagel)

                    definition_kato = kato_ranking_candidates(identifier, params)
                    definition_list_kato.append(definition_kato)

                    # if definition_pagel[0].definition != definition_kato[0].definition:

                    with open('test_result.txt', 'a') as f:
                        f.write(sep.join([identifier.text_tex,
                          # definition_pattern_based[0].definition,
                          f'{definition_pagel[0].score:.4f}',
                          definition_pagel[0].definition,
                          f'{definition_kato[0].score:.4f}',
                          definition_kato[0].definition]))
                        f.write('\n')

                (_, score_definition_pagel) = evaluate_identifier_definition(doc0.identifiers,
                           gold_identifier,
                           definition_list_pagel,
                           gold_definition_list)
                if score_definition_pagel[0] > score_pagel_recall_best[0][0]:
                    score_pagel_recall_best = [score_definition_pagel, params]
                if score_definition_pagel[1] > score_pagel_precision_best[0][1]:
                    score_pagel_precision_best = [score_definition_pagel, params]

                (_, score_definition_pagel) = evaluate_identifier_definition(doc0.identifiers,
                           gold_identifier,
                           definition_list_pagel,
                           gold_definition_list, max_rank=2)
                if score_definition_pagel[0] > score_pagel_recall_best_rank2[0][0]:
                    score_pagel_recall_best_rank2 = [score_definition_pagel, params]
                if score_definition_pagel[1] > score_pagel_precision_best_rank2[0][1]:
                    score_pagel_precision_best_rank2 = [score_definition_pagel, params]

                (_, score_definition_kato) = evaluate_identifier_definition(doc0.identifiers,
                           gold_identifier,
                           definition_list_kato,
                           gold_definition_list)
                if score_definition_kato[0] >= score_kato_recall_best[0][0]:
                    score_kato_recall_best = [score_definition_kato, params]
                    with open('test_result.txt', 'a') as f:
                        f.write(str(score_kato_recall_best))
                        f.write('\n')

                    # print(score_kato_recall_best)
                if score_definition_kato[1] >= score_kato_precision_best[0][1]:
                    score_kato_precision_best = [score_definition_kato, params]
                    with open('test_result.txt', 'a') as f:
                        f.write(str(score_kato_precision_best))
                        f.write('\n')
                    # print(score_kato_precision_best)

                (_, score_definition_kato) = evaluate_identifier_definition(doc0.identifiers,
                           gold_identifier,
                           definition_list_kato,
                           gold_definition_list, max_rank=2)
                if score_definition_kato[0] > score_kato_recall_best_rank2[0][0]:
                    score_kato_recall_best_rank2 = [score_definition_kato, params]
                if score_definition_kato[1] > score_kato_precision_best_rank2[0][1]:
                    score_kato_precision_best_rank2 = [score_definition_kato, params]




In [13]:
(score_identifier, score_definition_pattern) = evaluate_identifier_definition(doc0.identifiers,
                               gold_identifier,
                               definition_list_pattern,
                               gold_definition_list)
print('======================')
print(f'identifier: '
      f'recall={score_identifier[0]}, precision={score_identifier[1]}')
print('======================')
print(f'definition: '
      f'recall={score_definition_pattern[0]}, '
      f'precision={score_definition_pattern[1]}'
      )

identifier: recall=1.0, precision=0.8421052631578947
definition: recall=0.375, precision=0.375


In [ ]:
(_, score_definition_pagel) = evaluate_identifier_definition(doc0.identifiers,
                               gold_identifier,
                               definition_list_pagel,
                               gold_definition_list)
print('======================')
print(f'definition: '
      f'recall={score_definition_pagel[0]}, '
      f'precision={score_definition_pagel[1]}'
      )

In [ ]:
(_, score_definition_kato) = evaluate_identifier_definition(doc0.identifiers,
                               gold_identifier,
                               definition_list_kato,
                               gold_definition_list)
print(f'definition: '
      f'recall={score_definition_kato[0]}, '
      f'precision={score_definition_kato[1]}'
      )